<a href="https://colab.research.google.com/github/himanshu11235/TDG-Partner-Internship-/blob/master/TDG_1_web_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
from nltk.stem import WordNetLemmatizer
import requests
import re
import string
import math

In [31]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
#***** functions
# FUNCTIONAL FORM OF INDEX
def index(urls):
  urls = set(urls)
  BOWs = {}
  for url in urls:
    x = requests.get(url)
    pge_txt = x.text
    # untabify
    clean = re.compile('<.*?>')
    pge_txt = re.sub(clean, '', pge_txt)
    # normalize
    #lower
    pge_txt = pge_txt.lower()
    # removing numbers
    pge_txt = re.sub(r'\d', '', pge_txt)
    #removing special characters !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    to_remove = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'
    table = {ord(char): ' ' for char in to_remove}
    pge_txt = pge_txt.translate(table)
    #strip
    pge_txt = pge_txt.strip()
    #tokenize
    pge_wrd_lst = nltk.tokenize.word_tokenize(pge_txt)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    pge_wrd_lst = [i for i in pge_wrd_lst if i not in stop_words]
    BOW_dict = {}
    for i in pge_wrd_lst :
      if i in BOW_dict:
        pass
      else:
        BOW_dict[i] = pge_wrd_lst.count(i) 
    
    #lemmatization
    lemmetizer=WordNetLemmatizer()
    pge_lemmatized = [lemmetizer.lemmatize(i) for i in pge_wrd_lst]
    BOW_dict_lemmatized = {}
    for i in pge_lemmatized :
      if i in BOW_dict_lemmatized:
        pass
      else:
        BOW_dict_lemmatized[i] = pge_lemmatized.count(i) 
    
    BOWs[url] = [BOW_dict, BOW_dict_lemmatized]
  return BOWs


# FUNCTIONAL FORM OF QUERRY
def search(querry, BOWs):
  lemmatised_scores = {}
  non_lemmatised_scores = {}
  for link in BOWs:
    ## page vector dict
    bow = BOWs[link][0]
    bowl = BOWs[link][1]
    ## querry vector dict
    querry_dict = {}
    for i in querry :
      if i in querry_dict:
        pass
      else:
        querry_dict[i] = querry.count(i)  
    #non_lemmatized
    union = set(bow).union(set(querry_dict))
    L = len(union)
    dotp = 0
    for i in querry_dict:
      if i in bow :
        dotp += querry_dict[i]*bow[i]
      else:
        pass
    l1 = 0
    for i in bow:
      l1 = l1 + bow[i]*bow[i]
    l1 = math.sqrt(l1)
    l2 = 0
    for i in querry_dict:
      l2 = l2 + querry_dict[i]*querry_dict[i]
    l2 = math.sqrt(l2)
    cosine = float(dotp)/(l1*l2)
    non_lemmatised_scores[link] = cosine
    #lemmatized
    union = set(bowl).union(set(querry_dict))
    L = len(union)
    dotp = 0
    for i in querry_dict:
      if i in bowl :
        dotp += querry_dict[i]*bowl[i]
      else:
        pass
    l1 = 0
    for i in bowl:
      l1 = l1 + bowl[i]*bowl[i]
    l1 = math.sqrt(l1)
    l2 = 0
    for i in querry_dict:
      l2 = l2 + querry_dict[i]*querry_dict[i]
    l2 = math.sqrt(l2)
    cosine = float(dotp)/(l1*l2)
    lemmatised_scores[link] = cosine
  return [lemmatised_scores, non_lemmatised_scores]

####### FUNCTION FOR SORTING 
def f(element):
  return -element[1]


In [0]:
# CLASS CONTAINING INDEX AND QUERRY AND HENCE WHOLE SEARCH ENGINE
class websearchengine:
  def __init__(self):
    self.BOWs = {}

  def index(self, urls):
    urls = set(urls)
    self.BOWs = {}
    for url in urls:
      x = requests.get(url)
      pge_txt = x.text
      # untabify
      clean = re.compile('<.*?>')
      pge_txt = re.sub(clean, '', pge_txt)
      # normalize
      #lower
      pge_txt = pge_txt.lower()
      # removing numbers
      pge_txt = re.sub(r'\d', '', pge_txt)
      #removing special characters !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
      to_remove = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'
      table = {ord(char): ' ' for char in to_remove}
      pge_txt = pge_txt.translate(table)
      #strip
      pge_txt = pge_txt.strip()
      #tokenize
      pge_wrd_lst = nltk.tokenize.word_tokenize(pge_txt)
      stop_words = set(nltk.corpus.stopwords.words('english'))
      pge_wrd_lst = [i for i in pge_wrd_lst if i not in stop_words]
      BOW_dict = {}
      for i in pge_wrd_lst :
        if i in BOW_dict:
          pass
        else:
          BOW_dict[i] = pge_wrd_lst.count(i) 
      
      #lemmatization
      lemmetizer=WordNetLemmatizer()
      pge_lemmatized = [lemmetizer.lemmatize(i) for i in pge_wrd_lst]
      BOW_dict_lemmatized = {}
      for i in pge_lemmatized :
        if i in BOW_dict_lemmatized:
          pass
        else:
          BOW_dict_lemmatized[i] = pge_lemmatized.count(i) 
      
      self.BOWs[url] = [BOW_dict, BOW_dict_lemmatized]
    
  def search(self,querry):
    lemmatised_scores  =  {}
    non_lemmatised_scores = {}
    if(len(self.BOWs) == 0):
      print("No URLs to traverse")
      return
    for link in self.BOWs:
      ## page vector dict
      bow = self.BOWs[link][0]
      bowl = self.BOWs[link][1]
      ## querry vector dict
      querry_dict = {}
      for i in querry :
        if i in querry_dict:
          pass
        else:
          querry_dict[i] = querry.count(i)  
      #non_lemmatized
      union = set(bow).union(set(querry_dict))
      L = len(union)
      dotp = 0
      for i in querry_dict:
        if i in bow :
          dotp += querry_dict[i]*bow[i]
        else:
          pass
      l1 = 0
      for i in bow:
        l1 = l1 + bow[i]*bow[i]
      l1 = math.sqrt(l1)
      l2 = 0
      for i in querry_dict:
        l2 = l2 + querry_dict[i]*querry_dict[i]
      l2 = math.sqrt(l2)
      
      cosine = float(dotp)/(l1*l2)
      non_lemmatised_scores[link] = cosine
      #lemmatized
      union = set(bowl).union(set(querry_dict))
      L = len(union)
      dotp = 0
      for i in querry_dict:
        if i in bowl :
          dotp += querry_dict[i]*bowl[i]
        else:
          pass
      l1 = 0
      for i in bowl:
        l1 = l1 + bowl[i]*bowl[i]
      l1 = math.sqrt(l1)
      l2 = 0
      for i in querry_dict:
        l2 = l2 + querry_dict[i]*querry_dict[i]
      l2 = math.sqrt(l2)
      cosine = float(dotp)/(l1*l2)
      lemmatised_scores[link] = cosine
    lemmatised_scores = sorted(lemmatised_scores.items(), key = f)
    non_lemmatised_scores = sorted(non_lemmatised_scores.items(), key = f)
    return {'lemmatised_scores_dictionary' : lemmatised_scores, 'non_lemmatised_score_dictionary':non_lemmatised_scores}





In [36]:
urls = ['https://www.programiz.com/python-programming/methods/list/count', 'https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/','https://www.programiz.com/python-programming/methods/set/union', 'https://docs.python.org/3/library/math.html']
querries = ['and', 'web'] 
engine = websearchengine()
engine.index(urls)
sorted_scores_dict = engine.search(querries)
if sorted_scores_dict != None:
  ## OUTPUT USING LEMMATISATION
  lemmatised_scores_dictionary = sorted_scores_dict['lemmatised_scores_dictionary']
  print("OUTPUT USING LEMMATISATION")
  for i in lemmatised_scores_dictionary:
    print('LINK : ', i[0], 'COSINE SCORE : ', i[1])
  # OUTPUT WITHOUT USING LEMMATISATION
  non_lemmatised_score_dictionary = sorted_scores_dict['non_lemmatised_score_dictionary']
  print("OUTPUT WITHOUT USING LEMMATISATION")
  for i in non_lemmatised_score_dictionary:
    print('LINK : ', i[0], 'COSINE SCORE : ', i[1])


OUTPUT USING LEMMATISATION
LINK :  https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/ COSINE SCORE :  0.007362001982874519
LINK :  https://www.programiz.com/python-programming/methods/list/count COSINE SCORE :  0.0010394572356857514
LINK :  https://www.programiz.com/python-programming/methods/set/union COSINE SCORE :  0.0010279148200143544
LINK :  https://docs.python.org/3/library/math.html COSINE SCORE :  0.0
OUTPUT WITHOUT USING LEMMATISATION
LINK :  https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/ COSINE SCORE :  0.007546149041401722
LINK :  https://www.programiz.com/python-programming/methods/list/count COSINE SCORE :  0.0010478830333401758
LINK :  https://www.programiz.com/python-programming/methods/set/union COSINE SCORE :  0.001036441400823938
LINK :  https://docs.python.org/3/library/math.html COSINE SCORE :  0.0


PROBLEMS OBSERVED:
1. We are using term frequency for creating bag of  words. But this approach does not assume anything about the relative importance of the words in a webpage. 
for that we can use term frequency–inverse document frequency to create out bag of words. this might give more precise results.
2. we are searching solely on the basis of correlation between bag of words of a page and the querry. This does not capture semantics and which sometimes becomes important. 
for this we can use word2vec word embeddings to and calculate the similarity based on these vector representations of words rather than depending solely on the frequency of words.
3. The normalisation of the page is leading to some wierd words and hence we need to refine our methods.
For now I am thinking of creating a more rigorous library to strictly extract the text from a html page which will be built by taking reference of whole HTML and CSS and how different elements are actually created in a webpage.
   



References
https://docs.python.org/3/library/re.html
https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908